In [1]:
import pandas as pd
import os
from collections import Counter, OrderedDict
import gc

import numpy as np

SEED = 42

In [3]:
import numpy as np

In [2]:
train = pd.read_csv('./train/train.csv')
train = train.rename(columns={'order_completed_at': 'month'})
addresses = pd.read_csv('./misc/addresses.csv')
# sample = pd.read_csv('./sample_submission.csv', sep=';')

In [4]:
display(train.shape)
sorted(train['month'].unique())

(1159576, 3)

['2020-01', '2020-02', '2020-03', '2020-04', '2020-05', '2020-06', '2020-07']

In [5]:
# find how many user_ids has each phone_id
count_ids = addresses.groupby('phone_id')['id'].count()
count_ids = pd.DataFrame(count_ids).rename(columns={'id': 'count_id'})

In [6]:
train = train.merge(addresses, on='phone_id', how='left')
train = train.merge(pd.DataFrame(count_ids), on='phone_id')

In [7]:
del count_ids, addresses

In [8]:
# trains = []
# for i, month in enumerate(sorted(train['month'].unique())):
#     trains.append(train[train['month'] == month])

In [9]:
test_phone_id = pd.DataFrame(
    train[train['month'] == '2020-07']['phone_id'].unique(), 
    columns=['phone_id']).sample(frac = 0.01).values[:,0]
test_phone_id

array([ 74409, 304244, 155930, ..., 325725, 457748,  37253])

In [10]:
holdout = train[train['phone_id'].isin(test_phone_id)]
train = train[~train['phone_id'].isin(test_phone_id)]

In [11]:
len(holdout), len(train), len(holdout)/ len(train)

(89863, 9006040, 0.009978081376498438)

In [12]:
li = []
for filename in sorted(os.listdir('./shipments')):
    df = pd.read_csv(os.path.join('./shipments', filename))
    li.append(df)
    
shipments = pd.concat(li, axis=0)

In [13]:
del df, li

In [14]:
shipments = train[['phone_id','id']].drop_duplicates().merge(shipments, 
                            left_on='id', right_on='ship_address_id', how='right')

In [23]:
shipments['month'] = pd.to_datetime(shipments['order_created_at']).dt.month
shipments['year'] = pd.to_datetime(shipments['order_created_at']).dt.year

In [36]:
shipments = shipments[(shipments['year']==2020) | ((shipments['year']==2019) & (shipments['month']==12))]

In [68]:
shipments.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2234176 entries, 0 to 2260685
Data columns (total 24 columns):
phone_id              float64
id                    float64
user_id               int64
ship_address_id       int64
shipment_id           int64
order_id              int64
order_created_at      object
order_completed_at    object
shipment_starts_at    object
retailer              object
s.order_state         object
shipment_state        object
s.city_name           object
s.store_id            int64
total_cost            float64
rate                  int64
dw_kind               object
promo_total           float64
total_weight          int64
platform              object
os                    object
shipped_at            object
month                 int64
year                  int64
dtypes: float64(4), int64(9), object(11)
memory usage: 426.1+ MB


In [60]:
platforms = pd.DataFrame(shipments.groupby(['phone_id','month'])['platform'].value_counts()).unstack(level=2).fillna(0)
platforms.head()

platform     
platform            app  web
phone_id month              
2.0      1          0.0  1.0
         2          0.0  2.0
         3          0.0  1.0
3.0      3          0.0  2.0
         5          1.0  0.0

In [61]:
shipments['os'] = shipments['os'].fillna('other')
os = pd.DataFrame(shipments.groupby(['phone_id','month'])['os'].value_counts()).unstack(level=2).fillna(0)
os.head()

/home/y_ksenia/anaconda3/envs/ngwl/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


os                              
os             android  ios linux  mac other windows
phone_id month                                      
2.0      1         0.0  0.0   0.0  0.0   0.0     1.0
         2         0.0  0.0   0.0  0.0   0.0     2.0
         3         0.0  0.0   0.0  0.0   0.0     1.0
3.0      3         2.0  0.0   0.0  0.0   0.0     0.0
         5         1.0  0.0   0.0  0.0   0.0     0.0

In [65]:
dw_kind = pd.DataFrame(shipments.groupby(['phone_id','month'])['dw_kind'].value_counts()).unstack(level=2).fillna(0)
dw_kind.head()

dw_kind                        
dw_kind        courier express_delivery pickup
phone_id month                                
2.0      1         1.0              0.0    0.0
         2         2.0              0.0    0.0
         3         1.0              0.0    0.0
3.0      3         2.0              0.0    0.0
         5         1.0              0.0    0.0

In [67]:
weight = pd.DataFrame(shipments.groupby(['phone_id','month'])['total_weight'].sum())
weight.head()

total_weight
phone_id month              
2.0      1             38865
         2             63276
         3             40777
3.0      3             31265
         5             25106

In [ ]:
pd.DataFrame(shipments.groupby(['phone_id','month'])['platform'].value_counts()).unstack(level=2).fillna(0)